In [1]:
from forward import solve_forward
from simulation import Simulation
import numpy as np
import util
from net import EEGMLP

2021-12-09 01:33:23.070500: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-12-09 01:33:23.070530: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
fwd = solve_forward()

# load previous simulation
sim = util.load_object('/media/thanos/Big Data/Thanos/TUC/Thesis/nn_trained/mlp_model_3_sim.pkl')

# create the trained nn.
eegnet = EEGMLP(sim=sim)
eegnet.load_nn('/media/thanos/Big Data/Thanos/TUC/Thesis/nn_trained/mlp_model_3.h5')

Forward problem is solved.
Electrodes: (73, 3)
Dipoles: (50460, 3)
Leadfield: (73, 50460)


2021-12-09 01:43:55.424073: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-12-09 01:43:55.425126: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-12-09 01:43:55.426281: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (thanos): /proc/driver/nvidia/version does not exist
2021-12-09 01:43:55.492464: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-12-09 01:43:56.303159: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 206684160 exceeds 10

Loaded model in EEGMLP :
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Hidden-1 (Dense)            (None, 256)               18944     
                                                                 
 Hidden-2 (Dense)            (None, 512)               131584    
                                                                 
 Hidden-3 (Dense)            (None, 1024)              525312    
                                                                 
 Output (Dense)              (None, 50460)             51721500  
                                                                 
Total params: 52,397,340
Trainable params: 52,397,340
Non-trainable params: 0
_________________________________________________________________


In [3]:
eeg = np.load('/media/thanos/Big Data/Thanos/TUC/Thesis/sim_data/sim_type_2/eeg_big_sim_1.npy')
sources = np.load('/media/thanos/Big Data/Thanos/TUC/Thesis/sim_data/sim_type_2/sources_big_sim_1.npy')

In [4]:
nmse = eegnet.evaluate_nmse(eeg=eeg, sources=sources)
mse = eegnet.evaluate_mse(eeg=eeg, sources=sources)
print('Normilized MSE:', np.mean(nmse))
print('MSE:', np.mean(mse))

2021-12-09 01:45:07.206208: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 2018400000 exceeds 10% of free system memory.


Normilized MSE: 0.2861879505382205
MSE: 95.0307592433411
